# EDA: Triple EMA Trend Following Strategy

## 📊 Overview
This notebook demonstrates a trend following strategy using triple Exponential Moving Averages (EMA) for signal generation. It analyzes WLD-USDT price action with three different EMA periods to identify strong directional trends and minimize false signals through multi-timeframe confirmation.

## 🎯 Objectives
1. **Triple EMA Setup**: Implement fast (12), medium (100), and slow (300) EMA indicators
2. **Trend Identification**: Use EMA alignment to identify strong bullish/bearish trends
3. **Signal Generation**: Create buy/sell signals based on EMA hierarchy conditions
4. **Visualization**: Display price action with EMA overlays and signal analysis
5. **Strategy Validation**: Analyze signal quality and trend following effectiveness

## 📋 Prerequisites
- CLOB data source with cached candle data
- pandas-ta library for technical indicators
- WLD-USDT 1-minute cached data from Binance Perpetual
- Plotly for interactive multi-panel charts

## ⚠️ Important Notes
- **Trend Following**: This strategy works best in trending markets, may struggle in sideways conditions
- **EMA Hierarchy**: All three EMAs must align for signal generation (reduces false signals)
- **Lagging Indicator**: EMAs are lagging indicators, signals may come after trend establishment
- **Risk Management**: Consider using additional filters or stop-losses for risk control

## 📈 Expected Outputs
- Triple EMA trend analysis with signal markers
- Multi-panel charts showing price action, EMAs, and signal line
- Signal statistics and trend following performance metrics
- Interactive visualization for strategy evaluation

---

## 🔧 Research Context
**Target Strategy**: XGridT (Cross-Grid Trading with Trend confirmation)
This analysis provides foundational trend detection for advanced grid trading strategies.

In [ ]:
# 📦 Environment Setup and Data Source Initialization
# Configure environment and initialize CLOB data source for cached data access
import warnings

warnings.filterwarnings("ignore")  # Suppress non-critical warnings

import pandas_ta as ta  # Technical analysis library for EMA calculations
from core.data_sources import CLOBDataSource

# Initialize CLOB data source for accessing cached candle data
clob = CLOBDataSource()
print("✅ CLOB data source initialized")

In [ ]:
# ⚙️ Trading Parameters Configuration
# Define market data parameters for WLD-USDT analysis
CONNECTOR_NAME = "binance_perpetual"    # Binance Perpetual Futures
TRADING_PAIR = "WLD-USDT"              # Worldcoin/Tether trading pair
INTERVAL = "1m"                        # 1-minute timeframe

print(f"📊 Analysis Configuration:")
print(f"  • Exchange: {CONNECTOR_NAME}")
print(f"  • Trading Pair: {TRADING_PAIR}")
print(f"  • Timeframe: {INTERVAL}")
print(f"  • Strategy: Triple EMA Trend Following")

In [ ]:
# 📥 Load Cached Candle Data
# Retrieve pre-downloaded candle data from cache for analysis
clob.load_candles_cache()

# Find the specific candle data matching our parameters
candles = next(candle for candle in clob.candles_cache.values()
               if candle.trading_pair == TRADING_PAIR and
               candle.connector_name == CONNECTOR_NAME and
               candle.interval == INTERVAL)

print(f"✅ Loaded cached data for {TRADING_PAIR}")
print(f"📊 Dataset info: {len(candles.data)} candles")
print(f"⏰ Date range: {candles.data.index[0]} to {candles.data.index[-1]}")
print(f"💰 Price range: ${candles.data['low'].min():.4f} - ${candles.data['high'].max():.4f}")

In [ ]:
# 📈 Triple EMA Calculation and Initial Visualization
# Calculate three EMAs with different periods and generate basic chart
import plotly.graph_objects as go

# ============================================
# EMA CONFIGURATION
# ============================================
ema_short = 12      # Fast EMA - captures short-term trend changes
ema_medium = 100    # Medium EMA - filters out noise, confirms trend
ema_long = 300      # Slow EMA - identifies major trend direction

# Create working copy of data for analysis
df = candles.data.copy()

# Calculate EMAs using pandas-ta library
df.ta.ema(length=ema_short, append=True)
df.ta.ema(length=ema_medium, append=True)
df.ta.ema(length=ema_long, append=True)

# Extract EMA series for easier reference
short_ema = df[f"EMA_{ema_short}"]
medium_ema = df[f"EMA_{ema_medium}"]
long_ema = df[f"EMA_{ema_long}"]
close = df["close"]

print(f"📊 EMA Configuration:")
print(f"  • Fast EMA: {ema_short} periods (short-term momentum)")
print(f"  • Medium EMA: {ema_medium} periods (trend confirmation)")
print(f"  • Slow EMA: {ema_long} periods (major trend direction)")

# ============================================
# SIGNAL GENERATION LOGIC
# ============================================
# Bullish condition: Fast > Medium > Slow (all EMAs aligned upward)
long_condition = (short_ema > medium_ema) & (medium_ema > long_ema) & (short_ema > long_ema)

# Bearish condition: Fast < Medium < Slow (all EMAs aligned downward)
short_condition = (short_ema < medium_ema) & (medium_ema < long_ema) & (short_ema < long_ema)

# Initialize signal column and apply conditions
df["signal"] = 0
df.loc[long_condition, "signal"] = 1    # Buy signal
df.loc[short_condition, "signal"] = -1  # Sell signal

# ============================================
# INITIAL VISUALIZATION
# ============================================
# Create basic candlestick chart with EMAs
fig = candles.fig(width=1200)

# Add EMA lines with distinct colors
ema_fast = f'EMA_{ema_short}'
ema_med = f'EMA_{ema_medium}' 
ema_slow = f'EMA_{ema_long}'

fig.add_trace(go.Scatter(x=df.index, y=df[ema_fast],
                         line=dict(color='#00FF00', width=2),  # Green for fast
                         name='Fast EMA (12)'))
fig.add_trace(go.Scatter(x=df.index, y=df[ema_med],
                         line=dict(color='#FFA500', width=2),  # Orange for medium
                         name='Medium EMA (100)'))
fig.add_trace(go.Scatter(x=df.index, y=df[ema_slow],
                         line=dict(color='#0000FF', width=2),  # Blue for slow
                         name='Slow EMA (300)'))

# Update chart title and show
fig.update_layout(title=f"📊 {TRADING_PAIR} - Triple EMA Trend Analysis")

# Calculate signal statistics
buy_signals = (df['signal'] == 1).sum()
sell_signals = (df['signal'] == -1).sum()
print(f"\n📈 Signal Analysis:")
print(f"  • Buy signals: {buy_signals}")
print(f"  • Sell signals: {sell_signals}")
print(f"  • Total signals: {buy_signals + sell_signals}")

fig.show()

In [ ]:
# 📊 Dataset Overview
# Display the enhanced dataset with EMA indicators and signals
print(f"📋 Enhanced Dataset Overview:")
print(f"  • Total records: {len(df)}")
print(f"  • Columns: {len(df.columns)}")
print(f"  • New indicators: EMA_{ema_short}, EMA_{ema_medium}, EMA_{ema_long}, signal")
print(f"  • Signal distribution: {(df['signal'] != 0).sum()} active signals out of {len(df)} candles")

# Show sample of the data
df.head(10)

In [ ]:
# 🔄 Signal Generation Refinement
# Refine and validate the signal generation logic for accuracy
short_ema = df[f"EMA_{ema_short}"]
medium_ema = df[f"EMA_{ema_medium}"]
long_ema = df[f"EMA_{ema_long}"]
close = df["close"]

# ============================================
# SIGNAL CONDITIONS (Refined)
# ============================================
# Bullish condition: All EMAs must be in ascending order
# This indicates strong upward momentum across all timeframes
long_condition = (short_ema > medium_ema) & (medium_ema > long_ema) & (short_ema > long_ema)

# Bearish condition: All EMAs must be in descending order  
# This indicates strong downward momentum across all timeframes
short_condition = (short_ema < medium_ema) & (medium_ema < long_ema) & (short_ema < long_ema)

# Reset and apply refined signals
df["signal"] = 0
df.loc[long_condition, "signal"] = 1    # Buy signal (bullish alignment)
df.loc[short_condition, "signal"] = -1  # Sell signal (bearish alignment)

# ============================================
# SIGNAL VALIDATION
# ============================================
buy_signals = (df['signal'] == 1).sum()
sell_signals = (df['signal'] == -1).sum()
neutral_periods = (df['signal'] == 0).sum()

print(f"🎯 Refined Signal Analysis:")
print(f"  • Buy signals: {buy_signals} ({buy_signals/len(df)*100:.1f}%)")
print(f"  • Sell signals: {sell_signals} ({sell_signals/len(df)*100:.1f}%)")
print(f"  • Neutral periods: {neutral_periods} ({neutral_periods/len(df)*100:.1f}%)")
print(f"  • Signal coverage: {(buy_signals + sell_signals)/len(df)*100:.1f}% of total periods")

# Analyze signal transitions
signal_changes = (df['signal'].diff() != 0).sum()
print(f"  • Signal changes: {signal_changes} transitions")
print(f"  • Average signal duration: {len(df)/signal_changes:.1f} periods")

In [ ]:
# 📊 Comprehensive Multi-Panel Visualization
# Create detailed chart showing price action, EMAs, and signals
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create multi-row subplot layout
fig = make_subplots(
    rows=3, cols=1, 
    shared_xaxes=True, 
    vertical_spacing=0.02,
    subplot_titles=('💰 OHLC with Triple EMA System', '🎯 Signal Analysis', '📈 EMA Relationships'),
    row_heights=[0.6, 0.2, 0.2]
)

# ============================================
# TOP PANEL: PRICE ACTION WITH EMAS
# ============================================
# Add candlestick chart
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close'],
    name='Candlesticks'
), row=1, col=1)

# Add triple EMA system with clear color coding
ema_fast = f'EMA_{ema_short}'
ema_med = f'EMA_{ema_medium}' 
ema_slow = f'EMA_{ema_long}'

fig.add_trace(go.Scatter(
    x=df.index, y=df[ema_fast],
    line=dict(color='#00FF00', width=2),  # Green - Fast/Responsive
    name=f'Fast EMA ({ema_short})'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df.index, y=df[ema_med],
    line=dict(color='#FFA500', width=2),  # Orange - Medium/Filter
    name=f'Medium EMA ({ema_medium})'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df.index, y=df[ema_slow],
    line=dict(color='#0000FF', width=2),  # Blue - Slow/Trend
    name=f'Slow EMA ({ema_long})'
), row=1, col=1)

# ============================================
# MIDDLE PANEL: SIGNAL LINE
# ============================================
# Add signal line with color coding
signal_colors = ['red' if x == -1 else 'green' if x == 1 else 'gray' for x in df['signal']]

fig.add_trace(go.Scatter(
    x=df.index, y=df['signal'],
    mode='lines',
    name='Trading Signal',
    line=dict(color="white", width=2),
    fill='tonexty'
), row=2, col=1)

# Add signal markers for clarity
buy_points = df[df['signal'] == 1]
sell_points = df[df['signal'] == -1]

if not buy_points.empty:
    fig.add_trace(go.Scatter(
        x=buy_points.index, y=buy_points['signal'],
        mode='markers',
        marker=dict(color='green', size=8, symbol='triangle-up'),
        name='Buy Signals'
    ), row=2, col=1)

if not sell_points.empty:
    fig.add_trace(go.Scatter(
        x=sell_points.index, y=sell_points['signal'],
        mode='markers',
        marker=dict(color='red', size=8, symbol='triangle-down'),
        name='Sell Signals'
    ), row=2, col=1)

# ============================================
# BOTTOM PANEL: EMA SPREAD ANALYSIS
# ============================================
# Calculate EMA spreads for trend strength analysis
df['ema_spread_fast_med'] = df[ema_fast] - df[ema_med]
df['ema_spread_med_slow'] = df[ema_med] - df[ema_slow]

fig.add_trace(go.Scatter(
    x=df.index, y=df['ema_spread_fast_med'],
    mode='lines',
    name='Fast-Medium Spread',
    line=dict(color='#FF69B4', width=1)
), row=3, col=1)

fig.add_trace(go.Scatter(
    x=df.index, y=df['ema_spread_med_slow'],
    mode='lines',
    name='Medium-Slow Spread',
    line=dict(color='#9370DB', width=1)
), row=3, col=1)

# Add zero line for reference
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=3, col=1)

# ============================================
# LAYOUT CUSTOMIZATION
# ============================================
# Set consistent variables for chart title
exchange = CONNECTOR_NAME.replace('_', ' ').title()
trading_pair = TRADING_PAIR
interval = INTERVAL

fig.update_layout(
    title=f'🚀 {exchange} - {trading_pair} - {interval} | Triple EMA Trend Following Analysis',
    width=1500, height=1000,
    font=dict(color='#e1e1e1'),
    plot_bgcolor='#1e1e1e',
    paper_bgcolor='#1e1e1e',
    xaxis_rangeslider_visible=False,
    legend=dict(bgcolor='rgba(0,0,0,0)'),
    yaxis=dict(title='💰 Price (USDT)'),
    yaxis2=dict(title='📊 Signal', showgrid=False),
    yaxis3=dict(title='📈 EMA Spreads', showgrid=False),
    showlegend=True
)

# Update axes styling
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)

print(f"🎯 Final Analysis Summary:")
print(f"  • Strategy: Triple EMA Trend Following")
print(f"  • Total periods analyzed: {len(df)}")
print(f"  • Signal efficiency: {((buy_signals + sell_signals)/len(df)*100):.1f}% active signals")
print(f"  • Trend detection: EMA alignment method with {ema_short}/{ema_medium}/{ema_long} configuration")

# Show the comprehensive analysis
fig.show()